In [ ]:
'''
import cv2

def rotate_video(input_path, output_path, angle=90):
    # Open the input video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) if angle in [90, 270] else int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) if angle in [90, 270] else int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change codec if needed
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Rotate frame
        if angle == 90:
            frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            frame = cv2.rotate(frame, cv2.ROTATE_180)
        elif angle == 270:
            frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

        out.write(frame)

    cap.release()
    out.release()
    print(f"Rotated video saved as {output_path}")

    '''

In [6]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque

# Constants
MIN_KEYPOINT_CONFIDENCE = 0.1  # Minimum confidence for keypoints
RESCUER_PROXIMITY_THRESH = 0.3  # Relative distance threshold for rescuer detection

# Keypoint indices (YOLO pose model)
LEFT_SHOULDER = 5
RIGHT_SHOULDER = 6

# Global variables
chest_point = None  # (x, y) in image-space coordinates
firstTimeRotated = True
model = YOLO("yolo11n-pose.pt")

def detect_chest_landscape(frame):
    results = model(frame, verbose=False)
    if not results or len(results[0].keypoints.xy) == 0:
        return None

    keypoints = results[0].keypoints
    boxes = results[0].boxes
    confs = keypoints.conf  # shape: [num_persons, 17]

    for i, (kp, box) in enumerate(zip(keypoints.xyn, boxes.xywh)):
        x, y, w, h = box
        if h < w * 1.2:  # Not vertical enough
            continue

        if confs[i][LEFT_SHOULDER] < MIN_KEYPOINT_CONFIDENCE or confs[i][RIGHT_SHOULDER] < MIN_KEYPOINT_CONFIDENCE:
            continue

        left_shoulder = kp[LEFT_SHOULDER][:2].cpu().numpy()  # Convert tensor to numpy array
        right_shoulder = kp[RIGHT_SHOULDER][:2].cpu().numpy()  # Convert tensor to numpy array

        return (left_shoulder, right_shoulder)

    return None

def initialize_chest_detection(cap):
    global chest_point
    shoulder_samples = []
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    for i in range(15):
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose=False)
        if not results or len(results[0].keypoints.xy) == 0:
            continue

        debug_frame = results[0].plot()
        shoulders = detect_chest_landscape(frame)

        if shoulders:
            left_shoulder, right_shoulder = shoulders
            shoulder_samples.append((left_shoulder, right_shoulder))

            lx, ly = (left_shoulder * np.array([frame.shape[1], frame.shape[0]])).astype(int)
            rx, ry = (right_shoulder * np.array([frame.shape[1], frame.shape[0]])).astype(int)

            cv2.circle(debug_frame, (lx, ly), 5, (0, 0, 255), -1)
            cv2.circle(debug_frame, (rx, ry), 5, (0, 0, 255), -1)
            cv2.putText(debug_frame, f"CHEST pts - Frame {i+1}", (lx, ly - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        

        cv2.imshow("Chest Detection Debug", debug_frame)
        if cv2.waitKey(400) == ord('q'):
            break

    if shoulder_samples:
        avg_left = np.median([s[0] for s in shoulder_samples], axis=0)
        avg_right = np.median([s[1] for s in shoulder_samples], axis=0)

        avg_left_px = avg_left * np.array([frame.shape[1], frame.shape[0]])
        avg_right_px = avg_right * np.array([frame.shape[1], frame.shape[0]])

        midpoint = (avg_left_px + avg_right_px) / 2
        shoulder_dist = np.linalg.norm(avg_left_px - avg_right_px)

        # Move downward by 20% of shoulder distance (tunable)
        downward_offset = 0.4 * shoulder_dist
        chest_point = (int(midpoint[0]), int(midpoint[1] + downward_offset))
        print(f"Chest point initialized at: {chest_point}")
        cv2.circle(frame, chest_point, 8, (0, 55, 120), -1)
        cv2.putText(frame, "AYWA", (chest_point[0] + 5, chest_point[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    cv2.destroyWindow("Chest Detection Debug")

def find_rescuer(pose_results, frame_shape):
    if not pose_results or len(pose_results[0].boxes) == 0:
        return None

    boxes = pose_results[0].boxes.xywh.cpu().numpy()
    horizontal_objects = []

    for box in boxes:
        x, y, w, h = box
        if w > h * 1.5:
            horizontal_objects.append((x, y, w, h))

    if not horizontal_objects:
        return None

    people = []
    for i, box in enumerate(boxes):
        x, y, w, h = box
        if h > w * 1.2 and len(pose_results[0].keypoints) > i:
            people.append((i, x, y))

    if not people:
        return None

    min_distance = float('inf')
    rescuer_idx = None

    for (i, px, py) in people:
        for (hx, hy, hw, hh) in horizontal_objects:
            distance = np.sqrt(((px - hx) / frame_shape[1]) ** 2 + ((py - hy) / frame_shape[0]) ** 2)
            if distance < min_distance and distance < RESCUER_PROXIMITY_THRESH:
                min_distance = distance
                rescuer_idx = i

    return rescuer_idx

def process_frame(frame, width, height):
    global chest_point
    global firstTimeRotated
    
    rotated = False

    if width > height:
        # Rotate the frame 90 degrees clockwise to portrait mode
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        rotated = True

    # Process the frame using YOLO model
    results = model(frame, verbose=False)
    if not results:
        return frame

    annotated_frame = results[0].plot()

    if rotated and firstTimeRotated:
        # Adjust chest point for rotated frame
        if chest_point is not None:
            chest_point = (height - chest_point[1], chest_point[0])
            firstTimeRotated = False

    # Draw the chest point
    cv2.circle(annotated_frame, chest_point, 8, (255, 0, 0), -1)
    cv2.putText(annotated_frame, "CHEST", (chest_point[0] + 5, chest_point[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Find and draw rescuer
    rescuer_idx = find_rescuer(results, frame.shape[:2])
    if rescuer_idx is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        x1, y1, x2, y2 = boxes[rescuer_idx].astype(int)
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(annotated_frame, "RESCUER", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    return annotated_frame


def main():
    cap = cv2.VideoCapture("video_5.mp4")
    if not cap.isOpened():
        print("Error opening video file.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    initialize_chest_detection(cap)

    cv2.namedWindow("Rescue Analysis", cv2.WINDOW_NORMAL)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        output_frame = process_frame(frame, width, height)
        cv2.imshow("Rescue Analysis", output_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Chest point initialized at: (414, 137)
